<a href="https://colab.research.google.com/github/aliebi/transfer-learning-/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transfer learning 
in this kind of learning we use pretraind model for train network and we start by good values not a randomized value....

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

we test is gpu available or not

In [2]:
tf.test.is_gpu_available()

True

In [3]:
 tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
dataset = pd.read_csv('/movie_reviews.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


convert target of each document to 1 for pos and 0 for neg

In [6]:
dataset['sentiment'] = [1 if sentiment == 'positive' else 0 for sentiment in dataset['sentiment'].values]
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


divide dataset to test,train,development 

In [7]:

reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

train_reviews = reviews[:30000]
train_sentiments = sentiments[:30000]

val_reviews = reviews[30000:35000]
val_sentiments = sentiments[30000:35000]

test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
train_reviews.shape, val_reviews.shape, test_reviews.shape

((30000,), (5000,), (15000,))

In [8]:
!pip install contractions
!pip install BeautifulSoup4
!pip install textsearch 

     |████████████████████████████████| 317kB 6.6MB/s 
     |████████████████████████████████| 245kB 53.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81708 sha256=7d309b9e17e2c076ad6483e7e14c472eb959e45b1b6ffb6aa3b0876cf75b6fc6
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


we should  preprocces data before start

In [0]:

import contractions
from bs4 import BeautifulSoup
import unicodedata
import re

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    return contractions.fix(text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

def pre_process_document(document):
    # strip HTML
    document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    document = remove_accented_chars(document)
    # expand contractions    
    document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

divide data to train,test,dev

In [0]:
train_reviews = pre_process_corpus(train_reviews)
val_reviews = pre_process_corpus(val_reviews)
test_reviews = pre_process_corpus(test_reviews)

we should prepare data for give it to network...

In [0]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_reviews}, train_sentiments, 
    batch_size=256, num_epochs=None, shuffle=True)
    
# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_reviews}, train_sentiments, shuffle=False)
    
# Prediction on the whole validation set.
predict_val_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': val_reviews}, val_sentiments, shuffle=False)
    
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': test_reviews}, test_sentiments, shuffle=False)   

now we find preprocces model for our network that embed sentences ,you can read more about it here :https://tfhub.dev/google/universal-sentence-encoder/2

In [0]:
embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
    trainable=False)

define network architect

In [15]:
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=2,
          activation_fn=tf.nn.relu,
          dropout=0.1,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.005))

W0824 06:49:46.517555 140504106309504 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpnso5a_qr


train model and test validation and train data on it 

In [16]:
tf.logging.set_verbosity(tf.logging.ERROR)
import time

TOTAL_STEPS = 1500
STEP_SIZE = 100
for step in range(0, TOTAL_STEPS+1, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step =', step)
    start_time = time.time()
    dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
    elapsed_time = time.time() - start_time
    print('Train Time (s):', elapsed_time)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))


----------------------------------------------------------------------------------------------------
Training for step = 0
Train Time (s): 73.91745781898499
Eval Metrics (Train): {'accuracy': 0.84936666, 'accuracy_baseline': 0.5005, 'auc': 0.9282317, 'auc_precision_recall': 0.9282601, 'average_loss': 0.34399882, 'label/mean': 0.5005, 'loss': 43.914745, 'precision': 0.8569582, 'prediction/mean': 0.4940594, 'recall': 0.8390942, 'global_step': 100}
Eval Metrics (Validation): {'accuracy': 0.8454, 'accuracy_baseline': 0.505, 'auc': 0.9238464, 'auc_precision_recall': 0.9192813, 'average_loss': 0.3530658, 'label/mean': 0.495, 'loss': 44.133224, 'precision': 0.8401279, 'prediction/mean': 0.49856758, 'recall': 0.84929293, 'global_step': 100}

----------------------------------------------------------------------------------------------------
Training for step = 100
Train Time (s): 69.49799346923828
Eval Metrics (Train): {'accuracy': 0.85553336, 'accuracy_baseline': 0.5005, 'auc': 0.9326374, 'a

final result 

In [17]:
dnn.evaluate(input_fn=predict_train_input_fn)

{'accuracy': 0.8809,
 'accuracy_baseline': 0.5005,
 'auc': 0.9520035,
 'auc_precision_recall': 0.9516762,
 'average_loss': 0.281948,
 'global_step': 1600,
 'label/mean': 0.5005,
 'loss': 35.993362,
 'precision': 0.8895547,
 'prediction/mean': 0.48989645,
 'recall': 0.87006325}

In [18]:
dnn.evaluate(input_fn=predict_test_input_fn)

{'accuracy': 0.8670667,
 'accuracy_baseline': 0.5006667,
 'auc': 0.9410504,
 'auc_precision_recall': 0.9404416,
 'average_loss': 0.3109898,
 'global_step': 1600,
 'label/mean': 0.5006667,
 'loss': 39.532604,
 'precision': 0.87472826,
 'prediction/mean': 0.49171963,
 'recall': 0.857257}